In [6]:
# import re

# import time

# import math

# import requests

# import pandas as pd

# from bs4 import BeautifulSoup

# from pathlib import Path
 
# # ==============================

# # Config (GHS output)

# # ==============================

# NUMBEO_GHANA_URL_GHS = (

#     "https://www.numbeo.com/cost-of-living/country_result.jsp"

#     "?country=Ghana&displayCurrency=GHS"

# )

# WAYBACK_AVAILABLE = "https://archive.org/wayback/available"
 
# # Use mid-month targets for July to improve odds of a nearby snapshot

# TARGETS = {
#     "2023":"20240715",

#     "2024": "20240715",

#     "2025": "20250715",

# }
 
# OUT_XLSX = "numbeo_ghana_markets_cpi_GHS_2024_2025.xlsx"

# HEADERS = {"User-Agent": "Academic-Price-Research/1.0 (email@example.com)"}

# Path("wayback_html").mkdir(exist_ok=True)
 
# # ==============================

# # Wayback helpers

# # ==============================

# def closest_wayback(url: str, timestamp: str) -> str | None:

#     r = requests.get(

#         WAYBACK_AVAILABLE,

#         params={"url": url, "timestamp": timestamp},

#         headers=HEADERS,

#         timeout=30,

#     )

#     r.raise_for_status()

#     data = r.json()

#     snap = data.get("archived_snapshots", {}).get("closest")

#     return snap.get("url") if snap and snap.get("available") else None
 
# def fetch_html(url: str) -> str | None:

#     try:

#         r = requests.get(url, headers=HEADERS, timeout=45)

#         r.raise_for_status()

#         return r.text

#     except Exception:

#         return None
 
# # ==============================

# # Find & parse the "Markets" table robustly

# # ==============================

# MARKET_KEYWORDS = [

#     "Milk", "Bread", "Rice", "Eggs", "Cheese", "Chicken", "Beef",

#     "Apples", "Banana", "Oranges", "Tomato", "Potato", "Onion"

# ]
 
# def find_markets_table(soup: BeautifulSoup):

#     # 1) Try header text that contains "market" (case-insensitive)

#     for h in soup.find_all(["h1","h2","h3","h4"]):

#         if "market" in h.get_text(strip=True).lower():

#             tbl = h.find_next("table")

#             if tbl:

#                 return tbl

#     # 2) Fallback: scan all tables; pick the one whose first column contains many market-like items

#     candidate, best_hits = None, -1

#     for tbl in soup.find_all("table"):

#         hits = 0

#         for tr in tbl.find_all("tr"):

#             tds = tr.find_all(["td","th"])

#             if not tds:

#                 continue

#             left = tds[0].get_text(" ", strip=True)

#             if any(k.lower() in left.lower() for k in MARKET_KEYWORDS):

#                 hits += 1

#         if hits > best_hits:

#             best_hits = hits

#             candidate = tbl

#     # require some minimal hits to be confident

#     return candidate if best_hits >= 5 else None
 
# def extract_markets_table_ghs(html: str) -> list[dict]:

#     """

#     Parse the 'Markets' table in page order.

#     Returns list of dicts with: item, price_text (GHS line), price_value_ghs (float), range_text (if present).

#     """

#     soup = BeautifulSoup(html, "html.parser")

#     table = find_markets_table(soup)

#     if not table:

#         return []
 
#     out = []

#     for tr in table.find_all("tr"):

#         tds = tr.find_all(["td","th"])

#         if len(tds) < 2:

#             continue
 
#         item = tds[0].get_text(" ", strip=True)

#         right = tds[1].get_text(" ", strip=True)
 
#         # Heuristic: skip subheaders or empty rows

#         if not item or item.lower() in {"markets"}:

#             continue
 
#         # FIRST numeric token in the right cell is the displayed average (assumed in GHS)

#         m = re.search(r"([0-9]+(?:[.,][0-9]+)*)", right)

#         price_val = None

#         if m:

#             try:

#                 price_val = float(m.group(1).replace(",", ""))

#             except ValueError:

#                 price_val = None
 
#         # Optional min-max range

#         rmatch = re.search(r"([0-9][0-9,\.]*)\s*-\s*([0-9][0-9,\.]*)", right)

#         range_text = None

#         if rmatch:

#             range_text = f"{rmatch.group(1)} - {rmatch.group(2)}"
 
#         out.append(

#             {

#                 "item": item,

#                 "price_text": right,

#                 "price_value_ghs": price_val,

#                 "range_text": range_text,

#             }

#         )

#     # keep only rows with any numeric value to avoid headers

#     return [r for r in out if r["price_value_ghs"] is not None]
 
# # ==============================

# # Main: fetch snapshots, parse, CPI, Excel

# # ==============================

# records = []

# snap_meta = {}

# for year, ymd in TARGETS.items():

#     archived_url = closest_wayback(NUMBEO_GHANA_URL_GHS, ymd)

#     snap_meta[year] = archived_url

#     if not archived_url:

#         print(f"[WARN] No Wayback snapshot found for {year} at {NUMBEO_GHANA_URL_GHS}")

#         continue
 
#     html = fetch_html(archived_url)

#     if not html:

#         print(f"[WARN] Could not download archived HTML for {year}: {archived_url}")

#         continue
 
#     # Save HTML for inspection

#     Path(f"wayback_html/snapshot_{year}.html").write_text(html, encoding="utf-8")
 
#     rows = extract_markets_table_ghs(html)

#     if not rows:

#         print(f"[WARN] Parsed 0 'Markets' rows for {year}. Inspect: wayback_html/snapshot_{year}.html")

#     else:

#         print(f"[OK] Parsed {len(rows)} 'Markets' rows for {year} from {archived_url}")
 
#     for idx, row in enumerate(rows, start=1):

#         records.append(

#             {

#                 "snapshot_year": year,

#                 "order_on_page": idx,  # preserve exact order

#                 "archived_url": archived_url,

#                 **row,

#             }

#         )

#     time.sleep(1)  # be polite to the archive
 
# df = pd.DataFrame(records)
 
# if df.empty:

#     print("[ERROR] No rows parsed from any snapshot; cannot compute CPI.")

#     # still write a debug workbook with just the snapshot manifest

#     snap_df = pd.DataFrame([{"year": y, "archived_url": u} for y, u in snap_meta.items()])

#     with pd.ExcelWriter(OUT_XLSX, engine="xlsxwriter") as xl:

#         snap_df.to_excel(xl, sheet_name="snapshots", index=False)

#     raise SystemExit(1)
 
# # Wide format: one row per item with 2024/2025 columns in GHS

# pivot = (

#     df.pivot_table(

#         index=["order_on_page", "item"],

#         columns="snapshot_year",

#         values="price_value_ghs",

#         aggfunc="first",

#     )

#     .reset_index()

# )

# pivot.columns.name = None

# pivot = pivot.sort_values("order_on_page")
 
# # Items present in BOTH snapshots (for CPI)

# need_cols = [c for c in ["2024", "2025"] if c in pivot.columns]

# if need_cols != ["2024", "2025"]:

#     print(f"[ERROR] Missing columns in pivot: have {pivot.columns.tolist()}")

#     # Write what we have and exit gracefully

#     with pd.ExcelWriter(OUT_XLSX, engine="xlsxwriter") as xl:

#         df.to_excel(xl, sheet_name="markets_long_raw", index=False)

#         pivot.to_excel(xl, sheet_name="markets_wide_ghs", index=False)

#         pd.DataFrame([{"year": y, "archived_url": u} for y, u in snap_meta.items()]).to_excel(

#             xl, sheet_name="snapshots", index=False

#         )

#     raise SystemExit(1)
 
# both = pivot.dropna(subset=["2024", "2025"]).copy()
 
# # Equal-weights Laspeyres CPI: 2024 = 100

# if not both.empty:

#     base_sum = both["2024"].sum()

#     curr_sum = both["2025"].sum()

#     cpi_2024 = 100.0

#     cpi_2025 = (curr_sum / base_sum) * 100.0 if base_sum > 0 else math.nan

#     both["price_relative_%"] = (both["2025"] / both["2024"] - 1.0) * 100.0

# else:

#     base_sum = curr_sum = cpi_2024 = cpi_2025 = math.nan

#     print("[WARN] No overlapping items between 2024 and 2025; CPI cannot be computed.")
 
# # Snapshot manifest

# snapshots = pd.DataFrame([{"year": y, "archived_url": u} for y, u in snap_meta.items()])
 
# # Write Excel

# with pd.ExcelWriter(OUT_XLSX, engine="xlsxwriter") as xl:

#     df.to_excel(xl, sheet_name="markets_long_raw", index=False)

#     pivot.to_excel(xl, sheet_name="markets_wide_ghs", index=False)

#     if not both.empty:

#         both.to_excel(xl, sheet_name="matched_items", index=False)

#         pd.DataFrame(

#             {

#                 "metric": ["base_sum_2024_GHS", "curr_sum_2025_GHS", "CPI_2024=100", "CPI_2025"],

#                 "value": [base_sum, curr_sum, 100.0, cpi_2025],

#             }

#         ).to_excel(xl, sheet_name="summary", index=False)

#     snapshots.to_excel(xl, sheet_name="snapshots", index=False)
 
# print(f"Done. Wrote {OUT_XLSX}")

 

In [3]:
!pip install xlsxwriter

In [5]:
import re
import time
import math
import requests
import pandas as pd
from bs4 import BeautifulSoup
from pathlib import Path

# ==============================
# Config (GHS output)
# ==============================

NUMBEO_GHANA_URL_GHS = (
    "https://www.numbeo.com/cost-of-living/country_result.jsp"
    "?country=Ghana&displayCurrency=GHS"
)
WAYBACK_AVAILABLE = "https://archive.org/wayback/available"

# Use mid-month targets, plus fallbacks
TARGETS = {
    "2023": ["20230715", "20230801", "20230615"],
    "2024": ["20240715", "20240801", "20240615"],
    "2025": ["20250715", "20250801", "20250615"],
}

OUT_XLSX = "numbeo_ghana_markets_cpi_GHS_2024_2025.xlsx"
HEADERS = {"User-Agent": "Academic-Price-Research/1.0 (email@example.com)"}

Path("wayback_html").mkdir(exist_ok=True)

# ==============================
# Wayback helpers
# ==============================

def closest_wayback(url: str, timestamp: str) -> str | None:
    r = requests.get(
        WAYBACK_AVAILABLE,
        params={"url": url, "timestamp": timestamp},
        headers=HEADERS,
        timeout=30,
    )
    r.raise_for_status()
    data = r.json()
    snap = data.get("archived_snapshots", {}).get("closest")
    return snap.get("url") if snap and snap.get("available") else None

def fetch_html(url: str) -> str | None:
    try:
        r = requests.get(url, headers=HEADERS, timeout=45)
        r.raise_for_status()
        return r.text
    except Exception:
        return None

def find_best_snapshot(url: str, year: str, timestamps: list) -> str | None:
    for ts in timestamps:
        snap = closest_wayback(url, ts)
        if snap:
            print(f"[INFO] Using snapshot {ts} for {year}: {snap}")
            return snap
    print(f"[WARN] No snapshot found for {year} using any fallback dates.")
    return None

# ==============================
# Find & parse the "Markets" table
# ==============================

MARKET_KEYWORDS = [
    "Milk", "Bread", "Rice", "Eggs", "Cheese", "Chicken", "Beef",
    "Apples", "Banana", "Oranges", "Tomato", "Potato", "Onion"
]

def find_markets_table(soup: BeautifulSoup):
    for h in soup.find_all(["h1", "h2", "h3", "h4"]):
        if "market" in h.get_text(strip=True).lower():
            tbl = h.find_next("table")
            if tbl:
                return tbl
    candidate, best_hits = None, -1
    for tbl in soup.find_all("table"):
        hits = 0
        for tr in tbl.find_all("tr"):
            tds = tr.find_all(["td", "th"])
            if not tds:
                continue
            left = tds[0].get_text(" ", strip=True)
            if any(k.lower() in left.lower() for k in MARKET_KEYWORDS):
                hits += 1
        if hits > best_hits:
            best_hits = hits
            candidate = tbl
    return candidate if best_hits >= 5 else None

def extract_markets_table_ghs(html: str) -> list[dict]:
    soup = BeautifulSoup(html, "html.parser")
    table = find_markets_table(soup)
    if not table:
        return []
    out = []
    for tr in table.find_all("tr"):
        tds = tr.find_all(["td", "th"])
        if len(tds) < 2:
            continue
        item = tds[0].get_text(" ", strip=True)
        right = tds[1].get_text(" ", strip=True)
        if not item or item.lower() in {"markets"}:
            continue
        m = re.search(r"([0-9]+(?:[.,][0-9]+)*)", right)
        price_val = None
        if m:
            try:
                price_val = float(m.group(1).replace(",", ""))
            except ValueError:
                price_val = None
        rmatch = re.search(r"([0-9][0-9,\.]*)\s*-\s*([0-9][0-9,\.]*)", right)
        range_text = None
        if rmatch:
            range_text = f"{rmatch.group(1)} - {rmatch.group(2)}"
        out.append({
            "item": item,
            "price_text": right,
            "price_value_ghs": price_val,
            "range_text": range_text,
        })
    return [r for r in out if r["price_value_ghs"] is not None]

# ==============================
# Main Process
# ==============================

records = []
snap_meta = {}

for year, fallback_dates in TARGETS.items():
    archived_url = find_best_snapshot(NUMBEO_GHANA_URL_GHS, year, fallback_dates)
    snap_meta[year] = archived_url
    if not archived_url:
        continue

    html = fetch_html(archived_url)
    if not html:
        print(f"[WARN] Could not download archived HTML for {year}: {archived_url}")
        continue

    Path(f"wayback_html/snapshot_{year}.html").write_text(html, encoding="utf-8")

    rows = extract_markets_table_ghs(html)
    if not rows:
        print(f"[WARN] Parsed 0 'Markets' rows for {year}. Inspect: wayback_html/snapshot_{year}.html")
    else:
        print(f"[OK] Parsed {len(rows)} 'Markets' rows for {year} from {archived_url}")

    for idx, row in enumerate(rows, start=1):
        records.append({
            "snapshot_year": year,
            "order_on_page": idx,
            "archived_url": archived_url,
            **row,
        })

    time.sleep(1)

df = pd.DataFrame(records)

# Snapshot manifest
snapshots = pd.DataFrame([{"year": y, "archived_url": u} for y, u in snap_meta.items()])

# Write partial if empty
if df.empty:
    print("[ERROR] No data parsed. Exiting.")
    with pd.ExcelWriter(OUT_XLSX, engine="xlsxwriter") as xl:
        snapshots.to_excel(xl, sheet_name="snapshots", index=False)
    raise SystemExit(1)

# Pivot wide format
pivot = (
    df.pivot_table(
        index=["order_on_page", "item"],
        columns="snapshot_year",
        values="price_value_ghs",
        aggfunc="first",
    )
    .reset_index()
)
pivot.columns.name = None
pivot = pivot.sort_values("order_on_page")

# CPI if both years are available
need_cols = [c for c in ["2024", "2025"] if c in pivot.columns]
cpi_df = None
summary_df = None

if set(need_cols) == {"2024", "2025"}:
    both = pivot.dropna(subset=["2024", "2025"]).copy()
    if not both.empty:
        base_sum = both["2024"].sum()
        curr_sum = both["2025"].sum()
        cpi_2024 = 100.0
        cpi_2025 = (curr_sum / base_sum) * 100.0 if base_sum > 0 else math.nan
        both["price_relative_%"] = (both["2025"] / both["2024"] - 1.0) * 100.0
        cpi_df = both
        summary_df = pd.DataFrame({
            "metric": ["base_sum_2024_GHS", "curr_sum_2025_GHS", "CPI_2024=100", "CPI_2025"],
            "value": [base_sum, curr_sum, 100.0, cpi_2025],
        })
    else:
        print("[WARN] No overlapping items between 2024 and 2025; CPI not computed.")

# Write Excel
with pd.ExcelWriter(OUT_XLSX, engine="xlsxwriter") as xl:
    df.to_excel(xl, sheet_name="markets_long_raw", index=False)
    pivot.to_excel(xl, sheet_name="markets_wide_ghs", index=False)
    snapshots.to_excel(xl, sheet_name="snapshots", index=False)
    if cpi_df is not None:
        cpi_df.to_excel(xl, sheet_name="matched_items", index=False)
    if summary_df is not None:
        summary_df.to_excel(xl, sheet_name="summary", index=False)

print(f"✅ Done. Wrote {OUT_XLSX}")


[INFO] Using snapshot 20230715 for 2023: http://web.archive.org/web/20210614115541/https://www.numbeo.com/cost-of-living/country_result.jsp?country=Ghana&displayCurrency=GHS
[OK] Parsed 55 'Markets' rows for 2023 from http://web.archive.org/web/20210614115541/https://www.numbeo.com/cost-of-living/country_result.jsp?country=Ghana&displayCurrency=GHS
[INFO] Using snapshot 20240715 for 2024: http://web.archive.org/web/20210614115541/https://www.numbeo.com/cost-of-living/country_result.jsp?country=Ghana&displayCurrency=GHS
[OK] Parsed 55 'Markets' rows for 2024 from http://web.archive.org/web/20210614115541/https://www.numbeo.com/cost-of-living/country_result.jsp?country=Ghana&displayCurrency=GHS
[INFO] Using snapshot 20250801 for 2025: http://web.archive.org/web/20210614115541/https://www.numbeo.com/cost-of-living/country_result.jsp?country=Ghana&displayCurrency=GHS
[OK] Parsed 55 'Markets' rows for 2025 from http://web.archive.org/web/20210614115541/https://www.numbeo.com/cost-of-living/